## parameters

In [27]:
MODEL = "RotatE"
N_EPOCHS = 100
EMBEDDING_DIM = 400
BATCH_SIZE = 2000
LR = 0.02
NUM_NEGS_PER_POS = 50
BATCH_SIZE_EVAL = 8

DATA_FOLDER = "../data/interim/train_test_split/"


## pykeen

In [28]:
%load_ext autoreload
%autoreload 2

from pykeen.pipeline import pipeline

import sys
sys.path.append("..")

from src.cli.utils import triplesfactory_from_tsv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
train = triplesfactory_from_tsv(DATA_FOLDER + "train.csv")
test = triplesfactory_from_tsv(DATA_FOLDER + "test.csv")
val = triplesfactory_from_tsv(DATA_FOLDER + "val.csv")

result = pipeline(
    training=train,
    testing=test,
    validation=val,
    model=MODEL,
    model_kwargs=dict(embedding_dim=EMBEDDING_DIM),
    optimizer_kwargs=dict(lr=LR),
    training_kwargs=dict(
        num_epochs=N_EPOCHS,
        batch_size=BATCH_SIZE,
    ),
    negative_sampler_kwargs=dict(
        num_negs_per_pos=NUM_NEGS_PER_POS,
    ),
    evaluation_kwargs=dict(
        batch_size=BATCH_SIZE_EVAL,
    ),
    evaluator_kwargs=dict(filtered=True),
)

Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/1310 [00:00<?, ?batch/s]

KeyboardInterrupt: 

## dgl-ke

In [29]:
train.num_triples

2681508

In [30]:
# TODO: add other arguments used last time
tab = "\t"

dglke_command = f"DGLBACKEND=pytorch dglke_train --model_name {MODEL} --dataset heritageconnector --data_path '{DATA_FOLDER}' --delimiter '{tab}' \
--data_files train.csv val.csv test.csv --format raw_udd_hrt --max_step {int(train.num_triples/BATCH_SIZE*N_EPOCHS)} --test --valid --batch_size {BATCH_SIZE} \
--neg_sample_size {NUM_NEGS_PER_POS} --hidden_dim {EMBEDDING_DIM} --lr {LR} --neg_sample_size_eval {NUM_NEGS_PER_POS} --log_interval 100000 --batch_size_eval {BATCH_SIZE_EVAL} \
--save_path dglke_results/ --gpu 0 --no_eval_filter --mix_cpu_gpu --async_update"

print(dglke_command)

DGLBACKEND=pytorch dglke_train --model_name RotatE --dataset heritageconnector --data_path '../data/interim/train_test_split/' --delimiter '	' --data_files train.csv val.csv test.csv --format raw_udd_hrt --max_step 134075 --test --valid --batch_size 2000 --neg_sample_size 50 --hidden_dim 400 --lr 0.02 --neg_sample_size_eval 50 --log_interval 100000 --batch_size_eval 8 --save_path dglke_results/ --gpu 0 --no_eval_filter --mix_cpu_gpu --async_update
